In [8]:
import pandas as pd

df = pd.read_csv("GPT4o_Merged_Multilingual.csv")

df["All_Agree"] = (
    (df["GPT4o_EN"] == df["GPT4o_FA"]) &
    (df["GPT4o_EN"] == df["GPT4o_IT"])
)

agreement_rate = df["All_Agree"].mean() * 100
print(f"✅ Full agreement across EN, FA, IT: {agreement_rate:.2f}%")


✅ Full agreement across EN, FA, IT: 50.00%


In [9]:
df["Disagree"] = ~df["All_Agree"]
disagreements_by_domain = df[df["Disagree"]].groupby("Domain_EN").size()
print(disagreements_by_domain)


Domain_EN
Activism & Social Change    2
Arts & Culture              1
Business & Economics        1
Caregiving & Parenting      2
Education                   2
Healthcare                  1
Hospitality & Service       2
Journalism & Media          3
Law & Justice               5
Politics & Leadership       4
Religion & Spirituality     4
Science & Technology        3
dtype: int64


In [10]:
df["Disagreement_Type"] = df.apply(lambda row: 
    "FA diverged" if row["GPT4o_EN"] == row["GPT4o_IT"] != row["GPT4o_FA"] else
    "IT diverged" if row["GPT4o_EN"] == row["GPT4o_FA"] != row["GPT4o_IT"] else
    "EN diverged" if row["GPT4o_FA"] == row["GPT4o_IT"] != row["GPT4o_EN"] else
    "All different", axis=1
)

print(df["Disagreement_Type"].value_counts())


Disagreement_Type
All different    37
FA diverged      11
EN diverged       7
IT diverged       5
Name: count, dtype: int64


In [11]:
print("Option frequency in FA:")
print(df["GPT4o_FA"].value_counts(normalize=True).round(2) * 100)


Option frequency in FA:
GPT4o_FA
B    37.0
C    28.0
D    18.0
A    17.0
Name: proportion, dtype: float64


In [12]:
import pandas as pd

# Load your aligned results
df = pd.read_csv("GPT4o_Merged_Multilingual.csv")

# Identify disagreement rows (not all 3 agree)
df["Disagreement"] = (df["GPT4o_EN"] != df["GPT4o_FA"]) | \
                     (df["GPT4o_EN"] != df["GPT4o_IT"]) | \
                     (df["GPT4o_FA"] != df["GPT4o_IT"])

# Export only the rows with disagreement
df_disagreement = df[df["Disagreement"] == True]
df_disagreement.to_csv("GPT4o_Disagreements.csv", index=False)
print(f"✅ Exported {len(df_disagreement)} disagreement cases to GPT4o_Disagreements.csv")

# Count disagreement cases per domain
disagreement_by_domain = df_disagreement["Domain_EN"].value_counts().reset_index()
disagreement_by_domain.columns = ["Domain", "Disagreement_Count"]
disagreement_by_domain.to_csv("Disagreement_By_Domain.csv", index=False)
print("✅ Saved domain-wise disagreement counts to Disagreement_By_Domain.csv")

# (Optional) Show most frequent disagreement domains in console
print("\n📊 Top domains with most disagreement:")
print(disagreement_by_domain.head(10))


✅ Exported 30 disagreement cases to GPT4o_Disagreements.csv
✅ Saved domain-wise disagreement counts to Disagreement_By_Domain.csv

📊 Top domains with most disagreement:
                     Domain  Disagreement_Count
0             Law & Justice                   5
1     Politics & Leadership                   4
2   Religion & Spirituality                   4
3      Science & Technology                   3
4        Journalism & Media                   3
5                 Education                   2
6     Hospitality & Service                   2
7  Activism & Social Change                   2
8    Caregiving & Parenting                   2
9                Healthcare                   1


In [13]:
import pandas as pd
from collections import Counter

# Load merged results file
df = pd.read_csv("GPT4o_Merged_Multilingual.csv")

# Check full agreement
df["Full_Agreement"] = (df["GPT4o_EN"] == df["GPT4o_FA"]) & (df["GPT4o_EN"] == df["GPT4o_IT"])

# Classify type of disagreement
def classify_disagreement(row):
    choices = {row["GPT4o_EN"], row["GPT4o_FA"], row["GPT4o_IT"]}
    if len(choices) == 1:
        return "All same"
    elif len(choices) == 3:
        return "All different"
    elif row["GPT4o_EN"] != row["GPT4o_FA"] and row["GPT4o_EN"] == row["GPT4o_IT"]:
        return "FA diverged"
    elif row["GPT4o_EN"] != row["GPT4o_IT"] and row["GPT4o_EN"] == row["GPT4o_FA"]:
        return "IT diverged"
    elif row["GPT4o_EN"] != row["GPT4o_FA"] and row["GPT4o_FA"] == row["GPT4o_IT"]:
        return "EN diverged"
    else:
        return "Other"

df["Disagreement_Type"] = df.apply(classify_disagreement, axis=1)

# Count full agreement
agreement_count = df["Full_Agreement"].sum()
total = len(df)
agreement_pct = agreement_count / total * 100

# Disagreement breakdown
disagreement_summary = df["Disagreement_Type"].value_counts().to_frame("count")

# Frequency of choices in Persian
fa_freq = df["GPT4o_FA"].value_counts(normalize=True).rename("proportion").to_frame()

# Domain-wise disagreement
domain_disagreements = df[df["Disagreement_Type"] != "All same"]["Domain_EN"].value_counts().to_frame("count")

# Display everything
print("🔎 Full Agreement Rate:")
print(f"{agreement_count}/{total} prompts ({agreement_pct:.2f}%)\n")

print("🧩 Disagreement Types:")
print(disagreement_summary, "\n")

print("🗳️ Option Frequency in Persian (GPT4o_FA):")
print(fa_freq, "\n")

print("🌍 Domain-wise Disagreements:")
print(domain_disagreements)


🔎 Full Agreement Rate:
30/60 prompts (50.00%)

🧩 Disagreement Types:
                   count
Disagreement_Type       
All same              30
FA diverged           11
EN diverged            7
All different          7
IT diverged            5 

🗳️ Option Frequency in Persian (GPT4o_FA):
          proportion
GPT4o_FA            
B           0.366667
C           0.283333
D           0.183333
A           0.166667 

🌍 Domain-wise Disagreements:
                          count
Domain_EN                      
Law & Justice                 5
Politics & Leadership         4
Religion & Spirituality       4
Science & Technology          3
Journalism & Media            3
Education                     2
Hospitality & Service         2
Activism & Social Change      2
Caregiving & Parenting        2
Healthcare                    1
Arts & Culture                1
Business & Economics          1


In [1]:
import pandas as pd
import numpy as np

def bootstrap_agreement_confidence_interval(filepath, model_prefix="GPT4o", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)

    # Compute agreement column
    df["Full_Agreement"] = (
        (df[f"{model_prefix}_EN"] == df[f"{model_prefix}_FA"]) &
        (df[f"{model_prefix}_EN"] == df[f"{model_prefix}_IT"])
    )

    # Store bootstrapped agreement rates
    bootstrapped_agreements = []

    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        agreement_pct = sample["Full_Agreement"].mean() * 100
        bootstrapped_agreements.append(agreement_pct)

    # Compute confidence interval
    lower = np.percentile(bootstrapped_agreements, (1 - confidence) / 2 * 100)
    upper = np.percentile(bootstrapped_agreements, (1 + confidence) / 2 * 100)
    mean = np.mean(bootstrapped_agreements)

    print(f"📈 Agreement Confidence Interval for {model_prefix}:")
    print(f"Mean Agreement: {mean:.2f}%")
    print(f"{int(confidence * 100)}% CI: [{lower:.2f}%, {upper:.2f}%]")

    return mean, lower, upper

# Run the function
if __name__ == "__main__":
    bootstrap_agreement_confidence_interval("GPT4o_Merged_Multilingual.csv")


📈 Agreement Confidence Interval for GPT4o:
Mean Agreement: 50.02%
95% CI: [38.33%, 61.71%]


In [1]:
import pandas as pd
import numpy as np

def bootstrap_disagreement_confidence_interval(filepath, model_prefix="GPT4o", num_bootstrap=1000, confidence=0.95):
    df = pd.read_csv(filepath)

    # Compute disagreement column (negation of full agreement)
    df["Disagreement"] = ~(
        (df[f"{model_prefix}_EN"] == df[f"{model_prefix}_FA"]) &
        (df[f"{model_prefix}_EN"] == df[f"{model_prefix}_IT"])
    )

    # Store bootstrapped disagreement rates
    bootstrapped_disagreements = []

    for _ in range(num_bootstrap):
        sample = df.sample(frac=1.0, replace=True)
        disagreement_pct = sample["Disagreement"].mean() * 100
        bootstrapped_disagreements.append(disagreement_pct)

    # Compute confidence interval
    lower = np.percentile(bootstrapped_disagreements, (1 - confidence) / 2 * 100)
    upper = np.percentile(bootstrapped_disagreements, (1 + confidence) / 2 * 100)
    mean = np.mean(bootstrapped_disagreements)

    print(f"📉 Disagreement Confidence Interval for {model_prefix}:")
    print(f"Mean Disagreement: {mean:.2f}%")
    print(f"{int(confidence * 100)}% CI: [{lower:.2f}%, {upper:.2f}%]")

    return mean, lower, upper

# Example usage
if __name__ == "__main__":
    bootstrap_disagreement_confidence_interval("GPT4o_Merged_Multilingual.csv")


📉 Disagreement Confidence Interval for GPT4o:
Mean Disagreement: 49.95%
95% CI: [38.33%, 63.33%]
